### Experimental historical illustration basic categorization work

Experimental historical newspaper illustration basic categorization demo.

Acknowledgments
<img src="https://digi.kansalliskirjasto.fi/images/sosiaali_en.png" alt="European Regional Development Fund" height="110">

<img src="https://digi.kansalliskirjasto.fi/images/en_EU_rgb.png" alt="Leverage from the EU 2014-2020" height="110">


#### Setup 

Download the image classifier and the labels.

Note! the download links below will inactivate at December 2019, after which please download the packages (Illustration base type classifier model file & labels) from https://digi.kansalliskirjasto.fi/opendata and update links below.

In [5]:


!curl https://digi.kansalliskirjasto.fi/opendata-download-file/035f2c8bfc124d159b066272913c67e8/nlf_basetype_classifier.pb --output nlf_basetype_classifier.pb
    
!curl https://digi.kansalliskirjasto.fi/opendata-download-file/976df7ccbcef4831a270e6254263cfd6/nlf_basetype_classifier_labels.txt  --output nlf_basetype_classifier_labels.txt
    
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 49 83.4M   49 41.4M    0     0  45.1M      0  0:00:01 --:--:--  0:00:01 45.0M
100 83.4M  100 83.4M    0     0  52.0M      0  0:00:01  0:00:01 --:--:-- 52.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    77  100    77    0     0    636      0 --:--:-- --:--:-- --:--:--   636


###  Image categorization code

Basic imports

In [6]:
import tensorflow as tf
import sys
import glob
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# setup for label changes
usedstrings = {
    'piirros painokuva': 'piirrospainokuva',
    'teksti mainos': 'mainos',
    'roskat': 'muut'
}


In [7]:
# The function to do the categorization, the img parameter carries the filename

def luokittelekuva(img=sys.argv[1]):
    image_data = tf.gfile.GFile(img, 'rb').read()

    AIDIR = "./core/"
    image_path = "./tmp/"

    # Read in the image_data
    label_lines = [line.rstrip() for line
                   in tf.gfile.GFile(AIDIR + "nlf_basetype_classifier_labels.txt")]

    # Unpersists graph from file
    with tf.gfile.GFile(AIDIR + "nlf_basetype_classifier.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

        # Feed the image_data as input to the graph and get first prediction
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})

        # Sort to show labels of first prediction in order of confidence
        limit_score = 0.6
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

        max_score = 0
        max_id = 0
        for node_id in top_k:
            score = predictions[0][node_id]
            if(max_score < score):
                max_score = score
                max_id = node_id

        # print(score)
        # if score>limit_score: #and human_string=="roskat"+"_"+str(limit_score):
        # tarkkuus="%.5f" % score
        #dst =human_string+'/'+tarkkuus+'_'+image_path

        human_string = label_lines[max_id]  # +"_"+str(max_score)
        print('%s : %s (score = %.5f)' % (image_path, human_string, score))

        if human_string in usedstrings:
            human_string = usedstrings[human_string]  # little bit better.

        return (human_string, score)


One expected drawing type:

In [13]:

!curl https://digi.kansalliskirjasto.fi/sanomalehti/binding/1198629/articles/43569800/images/1437505?scale=1.0 --output kuva1.jpg


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 40245    0 40245    0     0   247k      0 --:--:-- --:--:-- --:--:--  247k


In [14]:
# The actual way to run the function

luo, score = luokittelekuva("kuva1.jpg") # sys.argv[1])


./tmp/ : piirros painokuva (score = 0.00913)


In [15]:
print("Category: {0}, score {1} ".format(luo, score)) 

Category: piirrospainokuva, score 0.00912799034267664 


Illustration of unknown type:

In [16]:
!curl https://digi.kansalliskirjasto.fi/aikakausi/binding/1355524/articles/78396775/images/120919660?scale=1.0 --output kuva2.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 87108    0 87108    0     0   603k      0 --:--:-- --:--:-- --:--:--  603k


In [17]:
luo, score = luokittelekuva("kuva2.jpg") # sys.argv[1])
print("Category: {0}, score {1} ".format(luo, score))


./tmp/ : piirros painokuva (score = 0.00012)
Category: piirrospainokuva, score 0.00012121222971472889 


In [18]:

!curl https://digi.kansalliskirjasto.fi/sanomalehti/binding/679866/articles/1544609/images/10576494?scale=1.0 --output kuva3.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  838k    0  838k    0     0  4738k      0 --:--:-- --:--:-- --:--:-- 4738k


In [19]:
# then not so succesfull classification...

luo, score = luokittelekuva("kuva3.jpg") 
print("Category: {0}, score {1} ".format(luo, score))


./tmp/ : kartat (score = 0.01916)
Category: kartat, score 0.019164446741342545 
